# Explore usage of PyBedtools for Marker Design I/O

See 

- http://bedtools.readthedocs.org/en/latest/
- http://pythonhosted.org/pybedtools/

### Explore usage with Fasta

Could possibly use this to get sequence of an amplicon

http://pythonhosted.org/pybedtools/autodocs/pybedtools.bedtool.BedTool.seq.html#pybedtools.bedtool.BedTool.seq

also consider other pure py tools https://pypi.python.org/pypi/pyfaidx

In [9]:
import sys
sys.executable

'/Users/johnmccallum/anaconda/envs/py3-r-env/bin/python3'

In [8]:
sys.path

['',
 '/Users/johnmccallum/anaconda/envs/py3-r-env/lib/python35.zip',
 '/Users/johnmccallum/anaconda/envs/py3-r-env/lib/python3.5',
 '/Users/johnmccallum/anaconda/envs/py3-r-env/lib/python3.5/plat-darwin',
 '/Users/johnmccallum/anaconda/envs/py3-r-env/lib/python3.5/lib-dynload',
 '/Users/johnmccallum/miniconda3/envs/py3-r-env/lib/python3.5/site-packages/setuptools-19.4-py3.5.egg',
 '/Users/johnmccallum/anaconda/envs/py3-r-env/lib/python3.5/site-packages',
 '/Users/johnmccallum/anaconda/envs/py3-r-env/lib/python3.5/site-packages/IPython/extensions',
 '/Users/johnmccallum/.ipython']

In [2]:
!pip freeze | grep pybedtools

pybedtools==0.7.6


In [28]:
import pybedtools
from pybedtools import BedTool

In [8]:
from pybedtools import example_filename

### Example from docs

In [9]:
a = BedTool("""
... chr1 1 10
... chr1 50 55""", from_string=True)
fasta = example_filename('test.fa')
a = a.sequence(fi=fasta)
print(open(a.seqfn).read())

>chr1:1-10
GATGAGTCT
>chr1:50-55
CCATC



### Do same with original test data

In [10]:
ls /Users/johnmccallum/Documents/pcr_marker_design/test/test-data/

384um_251453690362217.txt  targets                    targets.fasta              targets.gff


Need a genome file

In [31]:
cat ../test/test-data/targets.fasta.fai

k69_93535	1628	11	60	61
k69_98089	749	1678	60	61


In [30]:
pybedtools.chromsizes?

In [90]:
b= BedTool("k69_93535 25 255",from_string=True)
b=b.sequence(fi=fn)
print(b)

k69_93535	25	255



In [11]:
c=BedTool('/Users/johnmccallum/Documents/pcr_marker_design/test/test-data/targets.gff')

In [17]:
!cat /Users/johnmccallum/Documents/pcr_marker_design/test/test-data/targets

k69_93535:SAMTOOLS:SNP:1147
k69_93535:SAMTOOLS:SNP:1336
k69_98089:SAMTOOLS:SNP:30
k69_98089:SAMTOOLS:SNP:550
k69_98089:SAMTOOLS:SNP:625


In [18]:
t1=BedTool('k69_93535 1146 1147',from_string=True)

Get annotations within _max product size_

In [44]:
max_product_size=200
my_target_region=c.intersect(t1.slop(b=max_product_size,g='../test/test-data/targets.fasta.fai'))
print(my_target_region)

k69_93535	SAMTOOLS	SNP	1141	1142	999	.	.	ID=k69_93535:SAMTOOLS:SNP:1141;Variant_seq=T;Reference_seq=C;DP=2644;VDB=0.0374;AF1=0.1882;AC1=5;DP4=748,786,225,294;MQ=42;FQ=999;PV4=0.037,0,0.036,0.39
k69_93535	SAMTOOLS	SNP	1147	1148	999	.	.	ID=k69_93535:SAMTOOLS:SNP:1147;Variant_seq=G;Reference_seq=C;DP=2645;VDB=0.0371;AF1=0.3527;G3=0.2771,0.7229,6.934e-153;HWE=0.0248;AC1=8;DP4=733,804,447,519;MQ=42;FQ=999;PV4=0.51,0,0.027,1
k69_93535	SAMTOOLS	SNP	1336	1337	999	.	.	ID=k69_93535:SAMTOOLS:SNP:1336;Variant_seq=A;Reference_seq=G;DP=2211;VDB=0.0395;AF1=0.4583;G3=0.08329,0.9167,1.041e-76;HWE=0.000925;AC1=11;DP4=487,645,470,542;MQ=35;FQ=999;PV4=0.12,0,0.00016,0.36



Can use operator overloading to exclude our target

In [45]:
print(my_target_region - t1)

k69_93535	SAMTOOLS	SNP	1141	1142	999	.	.	ID=k69_93535:SAMTOOLS:SNP:1141;Variant_seq=T;Reference_seq=C;DP=2644;VDB=0.0374;AF1=0.1882;AC1=5;DP4=748,786,225,294;MQ=42;FQ=999;PV4=0.037,0,0.036,0.39
k69_93535	SAMTOOLS	SNP	1336	1337	999	.	.	ID=k69_93535:SAMTOOLS:SNP:1336;Variant_seq=A;Reference_seq=G;DP=2211;VDB=0.0395;AF1=0.4583;G3=0.08329,0.9167,1.041e-76;HWE=0.000925;AC1=11;DP4=487,645,470,542;MQ=35;FQ=999;PV4=0.12,0,0.00016,0.36



b.seq?

In [91]:
open(b.seqfn).read()

'>k69_93535:25-255\nAGTTTCTCGCCCTCTCCACAGCGCTTTGAAGCGCTTTTAATGGCAGCAGATGCCGCCTCAGATCTCCCAAACCGACCCATTCCTAACACTCTCAGAAGATCCGATTCCAATTCCGTTCTACTGAACAAATACGAGCTGGGCAAGCTCCTCGGCCATGGAAATTTCGCCAAGGTTTACCTCGCCCGCAACCTCGCCTCCAACGAAGAAGTCGCTATCAAAGTCTTCGACAA\n'

Will be working with design globals accesible as dict

In design mode we would like to  :

- load up a fasta
- load up a vcf
- create dicts for a given target from this
    - work with just sub-sequence of fasta i.e. target length +/- product size max slop
    - just work with variants in this region
    - Target interval which will be vcf record ref feature length
    - mask which will be all regions but the target

Try using test data from galaxy-pcr-markers and Hong Yang

Kiwifruit_pseudomolecule.fa.gz (needs to be bgzipped, not gzipped, and indexed with tabix)

Try with specific target in vcf file
```
Chr1	10082	.	A	C
```

(No name for this. How do we handle naming? Assume in many contexts these will have a name..)

**NB** The following works fine with uncompressed fasta, fails with bgzipped

In [156]:
my_target=BedTool("Chr1 9000 10000",from_string= True)
HY_ref='/Users/cfljam/Documents/Kiwifruit_pseudomolecule.fa'
my_target=my_target.sequence(fi=HY_ref)
open(my_target.seqfn).read()

'>Chr1:9000-10000\nTATAACCTGACTAACCATGAACCTGGGTAGAATTCCACTCCTCCACCAAATTTTTTAACTTAACCAAGCATAAATGAATCTGAACGGTTTTAGCCCCCAATTTATATCCTCGTCCACTAACAATAGAAGGCAATGATCAGACAAGTTTCTGTGTAGACCCTACAAAATCAATTTTGGGAAATTCTGTCATTCTCTATCAATAAGAACTCTATCCAACTTGCTCCAGCTTTCATTTTCTTGAAGATTCAACCAAGTGAATTTCCTAACACATAGTGGTAAGTCAACCAAATCCATAAAATGGATGAAATTGTTGAATTCCCTCGTGGCACTGGTGACCCTTGTACTCCCCTTTCTTTTTCCTATTCTTCTTATCTCATTAAGAAGCCTGTATTTAGTGTTACGTGGGGAATCCAAAAAATTGTTTCGGAAGAAGCTAGCGTCACACTACTTAAATGACATGATCTAGTGAGTCTAATCAGCTTCACAGGGCTGAAGCCCTTGCTTCTAGATGCTGTTTTATTTATTTTTCGCACTGTGCTATCTTCCATTTCTGATATCATTTGGTTTGTTGCAAAATCATTATCTAACCAATGTATATCGTACTGTCATCTAAATTACCATAAGAAAAGAAATGTAGAGATGCTTCATTAGATAGTATTGTAGAAAATAAACAAATCTCTTTGGAAGTTAATTTAGAAAGTAACTGAGGGAGTTTGGTCATTCTTGCAGTCAGGGTATGCTTTGTTTTTTACCTGGCTCAGGTGATGCCATTAGCGCAACGCTTGGAAAAGAATTTTGTGTTTGAATCTCTTAATTTTGTGCTGAGGTTATTATTGGTGAATAGCAATCTTTTACGGAAGTACTCTTCCAGATGAGGTGCCAAAAAGTGGAAAATTATAAAGGTACACCTCGAGGTTTATTTTTGCAAGGTGAAAAAATTCTTTGTTTCTCCTTGTATTTCTACAGGTTTTCTCTCTTAGT

**NOTE** that VCF location is 1-based, versus bed/py 0-based

so bedtools interval for our SNP is 10081-10082

In [157]:
my_snp=BedTool("Chr1 10080 10083",from_string= True).sequence(fi=HY_ref)
open(my_snp.seqfn).read()

'>Chr1:10080-10083\nGAT\n'

So...could get string for Primer 3 by reading second line of this tmp file

### Try with bedtools

In [146]:
%%bash
cat /Users/cfljam/Documents/test.bed
bedtools getfasta -fi /Users/cfljam/Documents/Kiwifruit_pseudomolecule.fa \
                    -bed /Users/cfljam/Documents/test.bed \
                    -fo test.fasta
head test.fasta

Chr1	9000	10000
>Chr1:9000-10000
TATAACCTGACTAACCATGAACCTGGGTAGAATTCCACTCCTCCACCAAATTTTTTAACTTAACCAAGCATAAATGAATCTGAACGGTTTTAGCCCCCAATTTATATCCTCGTCCACTAACAATAGAAGGCAATGATCAGACAAGTTTCTGTGTAGACCCTACAAAATCAATTTTGGGAAATTCTGTCATTCTCTATCAATAAGAACTCTATCCAACTTGCTCCAGCTTTCATTTTCTTGAAGATTCAACCAAGTGAATTTCCTAACACATAGTGGTAAGTCAACCAAATCCATAAAATGGATGAAATTGTTGAATTCCCTCGTGGCACTGGTGACCCTTGTACTCCCCTTTCTTTTTCCTATTCTTCTTATCTCATTAAGAAGCCTGTATTTAGTGTTACGTGGGGAATCCAAAAAATTGTTTCGGAAGAAGCTAGCGTCACACTACTTAAATGACATGATCTAGTGAGTCTAATCAGCTTCACAGGGCTGAAGCCCTTGCTTCTAGATGCTGTTTTATTTATTTTTCGCACTGTGCTATCTTCCATTTCTGATATCATTTGGTTTGTTGCAAAATCATTATCTAACCAATGTATATCGTACTGTCATCTAAATTACCATAAGAAAAGAAATGTAGAGATGCTTCATTAGATAGTATTGTAGAAAATAAACAAATCTCTTTGGAAGTTAATTTAGAAAGTAACTGAGGGAGTTTGGTCATTCTTGCAGTCAGGGTATGCTTTGTTTTTTACCTGGCTCAGGTGATGCCATTAGCGCAACGCTTGGAAAAGAATTTTGTGTTTGAATCTCTTAATTTTGTGCTGAGGTTATTATTGGTGAATAGCAATCTTTTACGGAAGTACTCTTCCAGATGAGGTGCCAAAAAGTGGAAAATTATAAAGGTACACCTCGAGGTTTATTTTTGCAAGGTGAAAAAATTCTTTGTTTCTCCTTGTATTTCTACAGG